In [12]:
import os
import cv2
import tensorflow as tf
import segmentation_models as sm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from dataset import modify_mask
import requests, io
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from dataset import create_dataset, map_function
import pathlib
import tensorflow_model_optimization as tfmot
from tensorflow import keras

In [3]:
# Recreate the exact same model, including its weights and the optimizer
basedir = os.getcwd()
model_path = os.path.join(basedir, 'result', 'tf_full.h5')
history_path = os.path.join(basedir, 'result', 'history.csv')
#with tf.device('/cpu:0'):
unet_model = tf.keras.models.load_model(model_path, custom_objects={"iou_score": sm.metrics.iou_score})

2023-06-03 15:03:47.512608: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-03 15:03:47.701914: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-03 15:03:47.702013: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-03 15:03:47.702833: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-03 15:03:47.702944: I tensorflow/compile

In [4]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
basedir = os.getcwd()
dpath = os.path.join(basedir, "input/")
df = pd.read_csv(dpath + 'metadata.csv')
df['Image'] = df['Image'].map(lambda x: dpath + 'Image/' + x)
df['Mask'] = df['Mask'].map(lambda x: dpath + 'Mask/' + x)
df_train, df_test = train_test_split(df, test_size=0.1)

In [13]:
batch_size = 1
epochs = 2
validation_split = 0.1
num_images = int(len(df_train) * (1 - validation_split))

end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                             final_sparsity=0.80,
                                                             begin_step=0,
                                                             end_step=end_step)
}

model_for_pruning = prune_low_magnitude(unet_model, **pruning_params)

model_for_pruning.compile(
    optimizer=keras.optimizers.Adam(learning_rate=2e-4),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[sm.metrics.iou_score],
)

model_for_pruning.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 prune_low_magnitude_Conv1_pad   (None, 65, 65, 3)   1           ['input_1[0][0]']                
 (PruneLowMagnitude)                                                                              
                                                                                                  
 prune_low_magnitude_Conv1 (Pru  (None, 32, 32, 32)  1730        ['prune_low_magnitude_Conv1_pad[0
 neLowMagnitude)                                                 ][0]']                           
                                                                                              

In [26]:
logdir = os.path.join(basedir, 'logs')
os.makedirs(logdir, exist_ok=True)
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

batch_size = 1
img_size = 64
#basedir = os.getcwd()
dpath = os.path.join(basedir, "input/")
df = pd.read_csv(dpath + 'metadata.csv')
df['Image'] = df['Image'].map(lambda x: dpath + 'Image/' + x)
df['Mask'] = df['Mask'].map(lambda x: dpath + 'Mask/' + x)

df_train, df_test = train_test_split(df, test_size=0.1)

train_df = df_train[:int(len(df_train) * (1 - validation_split))]
val_df = df_train[int(len(df_train) * (1 - validation_split)):]

train_dataset = create_dataset(train_df, batch_size, (img_size, img_size), training=True)
val_dataset = create_dataset(val_df, batch_size, (img_size, img_size), training=True)

# history = model.fit(
#         train_dataset, validation_data=test_dataset, epochs=args.epochs, callbacks=[es])

model_for_pruning.fit(train_dataset,
                  batch_size=batch_size, epochs=epochs, validation_data=val_dataset,
                  callbacks=callbacks)

2023-06-03 15:23:53.455479: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [234]
	 [[{{node Placeholder/_0}}]]
2023-06-03 15:23:53.455770: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [234]
	 [[{{node Placeholder/_0}}]]
2023-06-03 15:23:53.551649: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 17104896 exceeds 10% of free system memory.


Epoch 1/2


In [20]:
len(df_train)

261

In [25]:
len(val_df_)

27

In [17]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None))>